In [17]:
# Important library imports
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
from tensorflow.keras.preprocessing import image
from keras.preprocessing import image
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import Adam

In [18]:
# Read the labels.csv file and checking shape and records
labels_all = pd.read_csv(r"C:\Users\pj\Desktop\End to End Project\Dog_breed_prediction_Flask_App\artifacts\data_ingestion\labels.csv")
print(labels_all.shape)
labels_all.head()

(10222, 2)


,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [19]:
# Visualize the number of each breeds
breeds_all = labels_all["breed"]
breed_counts = breeds_all.value_counts()
breed_counts

breed
scottish_deerhound      126
maltese_dog             117
afghan_hound            116
entlebucher             115
bernese_mountain_dog    114
                       ... 
golden_retriever         67
brabancon_griffon        67
komondor                 67
eskimo_dog               66
briard                   66
Name: count, Length: 120, dtype: int64

In [22]:
# Selecting first 3 breeds (Limitation due to computation power)
CLASS_NAMES = ['golden_retriever','maltese_dog','bernese_mountain_dog']
labels = labels_all[(labels_all['breed'].isin(CLASS_NAMES))]
labels = labels.reset_index()
labels.head()

,index,id,breed
0,4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever
1,12,00693b8bc2470375cc744a6391d397ec,maltese_dog
2,25,00a366d4b4a9bbb6c8a63126697b7656,golden_retriever
3,37,0100f55e4f0fe28f2c0465d3fc4b9897,golden_retriever
4,90,022b34fd8734b39995a9f38a4f3e7b6b,maltese_dog


In [23]:
# Creating numpy matrix with zeros
X_data = np.zeros((len(labels), 224, 224, 3), dtype='float32')
# One hot encoding
Y_data = label_binarize(labels['breed'], classes = CLASS_NAMES)

# Reading and converting image to numpy array and normalizing dataset
for i in tqdm(range(len(labels))):
    image_path = 'C:/Users/pj/Desktop/End to End Project/Dog_breed_prediction_Flask_App/artifacts/data_ingestion/test/%s.jpg' % labels['id'][i]
    if not os.path.exists(image_path):
        print(f"Image not found: {image_path}")
        continue

    img = image.load_img(image_path, target_size=(224, 224))
    img = image.img_to_array(img)
    x = np.expand_dims(img.copy(), axis=0)
    X_data[i] = x / 255.0
    
# Printing train image and one hot encode shape & size
print('\nTrain Images shape: ',X_data.shape,' size: {:,}'.format(X_data.size))
print('One-hot encoded output shape: ',Y_data.shape,' size: {:,}'.format(Y_data.size))

100%|██████████| 298/298 [00:00<00:00, 1711.53it/s]

Image not found: C:/Users/pj/Desktop/End to End Project/Dog_breed_prediction_Flask_App/artifacts/data_ingestion/test/0021f9ceb3235effd7fcde7f7538ed62.jpg
Image not found: C:/Users/pj/Desktop/End to End Project/Dog_breed_prediction_Flask_App/artifacts/data_ingestion/test/00693b8bc2470375cc744a6391d397ec.jpg
Image not found: C:/Users/pj/Desktop/End to End Project/Dog_breed_prediction_Flask_App/artifacts/data_ingestion/test/00a366d4b4a9bbb6c8a63126697b7656.jpg
Image not found: C:/Users/pj/Desktop/End to End Project/Dog_breed_prediction_Flask_App/artifacts/data_ingestion/test/0100f55e4f0fe28f2c0465d3fc4b9897.jpg
Image not found: C:/Users/pj/Desktop/End to End Project/Dog_breed_prediction_Flask_App/artifacts/data_ingestion/test/022b34fd8734b39995a9f38a4f3e7b6b.jpg
Image not found: C:/Users/pj/Desktop/End to End Project/Dog_breed_prediction_Flask_App/artifacts/data_ingestion/test/02d54f0dfb40038765e838459ae8c956.jpg
Image not found: C:/Users/pj/Desktop/End to End Project/Dog_breed_prediction

In [24]:
# Building the Model
model = Sequential()

model.add(Conv2D(filters = 64, kernel_size = (5,5), activation ='relu', input_shape = (224,224,3)))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters = 32, kernel_size = (3,3), activation ='relu', kernel_regularizer = 'l2'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters = 16, kernel_size = (7,7), activation ='relu', kernel_regularizer = 'l2'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters = 8, kernel_size = (5,5), activation ='relu', kernel_regularizer = 'l2'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(128, activation = "relu", kernel_regularizer = 'l2'))
model.add(Dense(64, activation = "relu", kernel_regularizer = 'l2'))
model.add(Dense(len(CLASS_NAMES), activation = "softmax"))

model.compile(loss = 'categorical_crossentropy', optimizer = Adam(0.0001),metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 220, 220, 64)      4864      
                                                                 
 max_pooling2d (MaxPooling2  (None, 110, 110, 64)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 108, 108, 32)      18464     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 54, 54, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 48, 48, 16)        25104     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 24, 24, 16)        0

In [25]:
# Splitting the data set into training and testing data sets
X_train_and_val, X_test, Y_train_and_val, Y_test = train_test_split(X_data, Y_data, test_size = 0.1)
# Splitting the training data set into training and validation data sets
X_train, X_val, Y_train, Y_val = train_test_split(X_train_and_val, Y_train_and_val, test_size = 0.2)

In [26]:
# Training the model
epochs = 100
batch_size = 128

history = model.fit(X_train, Y_train, batch_size = batch_size, epochs = epochs, 
                    validation_data = (X_val, Y_val))

Epoch 1/100
2/2 [==============================] - 23s 8s/step - loss: 4.8994 - accuracy: 0.2570 - val_loss: 4.8791 - val_accuracy: 0.3333
Epoch 2/100
2/2 [==============================] - ETA: 0s - loss: 4.8739 - accuracy: 0.4019

KeyboardInterrupt: 

In [27]:
# Plot the training history
plt.figure(figsize=(12, 5))
plt.plot(history.history['accuracy'], color='r')
plt.plot(history.history['val_accuracy'], color='b')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'val'])


NameError: name 'history' is not defined

<Figure size 1200x500 with 0 Axes>

In [ ]:
Y_pred = model.predict(X_test)
score = model.evaluate(X_test, Y_test)
print('Accuracy over the test set: \n ', round((score[1]*100), 2), '%')

In [ ]:
# Plotting image to compare
plt.imshow(X_test[1,:,:,:])
plt.show()

# Finding max value from predition list and comaparing original value vs predicted
print("Originally : ",labels['breed'][np.argmax(Y_test[1])])
print("Predicted : ",labels['breed'][np.argmax(Y_pred[1])])